In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
label_encoder = LabelEncoder()

In [ ]:
train_feature= pd.read_csv('train_features.csv')
train_labels= pd.read_csv('train_labels.csv')
test_feature= pd.read_csv('test_features.csv')
submission= pd.read_csv('submission_format.csv')

In [ ]:
train_feature.drop(columns=['jumlah_anak_balita','jumlah_anak_remaja','tanggal_menjadi_anggota', 'keluhan'], inplace=True)

In [ ]:
train_feature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3817 entries, 0 to 3816
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tahun_kelahiran    3817 non-null   int64  
 1   pendidikan         3628 non-null   object 
 2   status_pernikahan  3605 non-null   object 
 3   pendapatan         3627 non-null   float64
 4   terakhir_belanja   3645 non-null   float64
 5   belanja_buah       3636 non-null   float64
 6   belanja_daging     3639 non-null   float64
 7   belanja_ikan       3624 non-null   float64
 8   belanja_kue        3603 non-null   float64
 9   pembelian_diskon   3639 non-null   float64
 10  pembelian_web      3652 non-null   float64
 11  pembelian_toko     3648 non-null   float64
dtypes: float64(9), int64(1), object(2)
memory usage: 358.0+ KB


In [ ]:
train_feature.isnull().sum()

tahun_kelahiran        0
pendidikan           189
status_pernikahan    212
pendapatan           190
terakhir_belanja     172
belanja_buah         181
belanja_daging       178
belanja_ikan         193
belanja_kue          214
pembelian_diskon     178
pembelian_web        165
pembelian_toko       169
dtype: int64

In [ ]:
train_feature.head(100)

,tahun_kelahiran,pendidikan,status_pernikahan,pendapatan,terakhir_belanja,belanja_buah,belanja_daging,belanja_ikan,belanja_kue,pembelian_diskon,pembelian_web,pembelian_toko
0,1979,Sarjana,Rencana Menikah,NaN,NaN,50575.0,260967.0,50575.0,20230.0,2.0,2.0,5.0
1,1950,Sarjana,Rencana Menikah,84063000.0,70.0,6069.0,44506.0,80920.0,20230.0,9.0,6.0,4.0
2,1966,Sarjana,Menikah,127532564.0,45.0,117611.0,265460.0,96341.0,145573.0,1.0,1.0,7.0
3,1961,Magister,Rencana Menikah,165579620.0,90.0,206346.0,1613901.0,27725.0,125868.0,0.0,7.0,8.0
4,1970,Sarjana,Rencana Menikah,117703159.0,78.0,90563.0,311757.0,40358.0,33875.0,7.0,6.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1954,Sarjana,Menikah,136676591.0,87.0,196142.0,NaN,122599.0,159239.0,3.0,11.0,5.0
96,1959,Magister,Menikah,165583000.0,43.0,291312.0,236691.0,449106.0,295358.0,0.0,3.0,9.0
97,1966,Sarjana,NaN,187041788.0,31.0,166150.0,1632669.0,312792.0,145162.0,2.0,4.0,4.0
98,1968,Sarjana,Sendiri,99416000.0,23.0,10115.0,109242.0,0.0,18207.0,6.0,7.0,2.0


In [ ]:
train_feature ['pendidikan'] = label_encoder.fit_transform(train_feature['pendidikan'])
train_feature ['status_pernikahan'] = label_encoder.fit_transform(train_feature['status_pernikahan'])

train_feature ['pendidikan'] = label_encoder.fit_transform(train_feature['pendidikan'])
train_feature ['status_pernikahan'] = label_encoder.fit_transform(train_feature['status_pernikahan'])

In [ ]:
columns_to_encode = ['pendidikan', 'status_pernikahan']

for column in columns_to_encode:
    # Menyimpan indeks di mana nilai NaN muncul
    nan_indices = train_feature[column].isnull()

    # Mengisi nilai NaN dengan string unik untuk menghindari kesalahan LabelEncoder
    train_feature[column].fillna('NaN', inplace=True)

    # Label encode kolom
    train_feature[column] = label_encoder.fit_transform(train_feature[column])

    # Kembalikan nilai NaN ke posisi semula
    train_feature[column] = np.where(nan_indices, np.nan, train_feature[column])

In [ ]:
train_feature.describe()

In [ ]:
imputer = KNNImputer(n_neighbors=5)
train_feature_imputed = imputer.fit_transform(train_feature)

In [ ]:
train_feature = pd.DataFrame(train_feature_imputed, columns = train_feature.columns)

In [ ]:
train_feature.head()

,tahun_kelahiran,pendidikan,status_pernikahan,pendapatan,terakhir_belanja,belanja_buah,belanja_daging,belanja_ikan,belanja_kue,pembelian_diskon,pembelian_web,pembelian_toko
0,1979.0,6.0,5.0,107028212.4,27.2,50575.0,260967.0,50575.0,20230.0,2.0,2.0,5.0
1,1950.0,6.0,5.0,84063000.0,70.0,6069.0,44506.0,80920.0,20230.0,9.0,6.0,4.0
2,1966.0,6.0,3.0,127532564.0,45.0,117611.0,265460.0,96341.0,145573.0,1.0,1.0,7.0
3,1961.0,2.0,5.0,165579620.0,90.0,206346.0,1613901.0,27725.0,125868.0,0.0,7.0,8.0
4,1970.0,6.0,5.0,117703159.0,78.0,90563.0,311757.0,40358.0,33875.0,7.0,6.0,5.0


In [ ]:
kolom_numerik_nan = train_feature.select_dtypes(exclude=['object']).columns[train_feature.select_dtypes(exclude=['object']).isnull().any()]

In [ ]:
imputer = KNNImputer(n_neighbors=5)
train_feature[kolom_numerik_nan] = imputer.fit_transform(train_feature[kolom_numerik_nan])

In [ ]:
train_feature.isnull().sum()

tahun_kelahiran      0
pendidikan           0
status_pernikahan    0
pendapatan           0
terakhir_belanja     0
belanja_buah         0
belanja_daging       0
belanja_ikan         0
belanja_kue          0
pembelian_diskon     0
pembelian_web        0
pembelian_toko       0
dtype: int64

In [ ]:
# 1. Identifikasi kolom-kolom kategorikal dan kolom-kolom numerik
kolom_kategorikal = train_feature.select_dtypes(include=['object']).columns
kolom_numerik = train_feature.select_dtypes(exclude=['object']).columns

In [ ]:
# 2. Pisahkan data menjadi dua bagian
data_train = train_feature.dropna(subset=kolom_kategorikal)
data_test = train_feature[train_feature[kolom_kategorikal].isnull().any(axis=1)]

In [ ]:
data_test.isnull().sum()

tahun_kelahiran        0
pendidikan           189
status_pernikahan    212
pendapatan             0
terakhir_belanja       0
belanja_buah           0
belanja_daging         0
belanja_ikan           0
belanja_kue            0
pembelian_diskon       0
pembelian_web          0
pembelian_toko         0
keluhan                0
dtype: int64

In [ ]:
# 3. Gunakan data dengan nilai yang tidak hilang untuk melatih model
X_train = data_train[kolom_numerik]
y_train = data_train[kolom_kategorikal]

In [ ]:
# 4. Gunakan model yang dilatih untuk memprediksi nilai-nilai yang hilang
model = RandomForestClassifier()  # Anda dapat menggunakan model yang lain juga
model.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
X_test = data_test[kolom_numerik]
nilai_prediksi = model.predict(X_test)

In [ ]:
# 5. Isi nilai-nilai yang hilang dengan nilai-nilai yang diprediksi
train_feature.loc[train_feature[kolom_kategorikal].isnull().any(axis=1), kolom_kategorikal] = nilai_prediksi

In [ ]:
train_feature.isnull().sum()

tahun_kelahiran      0
pendidikan           0
status_pernikahan    0
pendapatan           0
terakhir_belanja     0
belanja_buah         0
belanja_daging       0
belanja_ikan         0
belanja_kue          0
pembelian_diskon     0
pembelian_web        0
pembelian_toko       0
dtype: int64

In [ ]:
train_feature.head()

,tahun_kelahiran,pendidikan,status_pernikahan,pendapatan,terakhir_belanja,belanja_buah,belanja_daging,belanja_ikan,belanja_kue,pembelian_diskon,pembelian_web,pembelian_toko
0,1979.0,6.0,5.0,107028212.4,27.2,50575.0,260967.0,50575.0,20230.0,2.0,2.0,5.0
1,1950.0,6.0,5.0,84063000.0,70.0,6069.0,44506.0,80920.0,20230.0,9.0,6.0,4.0
2,1966.0,6.0,3.0,127532564.0,45.0,117611.0,265460.0,96341.0,145573.0,1.0,1.0,7.0
3,1961.0,2.0,5.0,165579620.0,90.0,206346.0,1613901.0,27725.0,125868.0,0.0,7.0,8.0
4,1970.0,6.0,5.0,117703159.0,78.0,90563.0,311757.0,40358.0,33875.0,7.0,6.0,5.0


In [ ]:
train_labels.head()

,jumlah_promosi
0,2
1,0
2,1
3,4
4,4


In [ ]:
test_feature.head()

,ID,tahun_kelahiran,pendidikan,status_pernikahan,pendapatan,jumlah_anak_balita,jumlah_anak_remaja,terakhir_belanja,belanja_buah,belanja_daging,belanja_ikan,belanja_kue,pembelian_diskon,pembelian_web,pembelian_toko,keluhan,tanggal_menjadi_anggota
0,2241,1957,Sarjana,NaN,120660151.0,0.0,1.0,63.0,122277.0,541399.0,214192.0,84305.0,4.0,6.0,10.0,0.0,NaN
1,2274,1968,Doktor,Menikah,163551821.0,0.0,1.0,58.0,35761.0,353335.0,63365.0,41112.0,2.0,5.0,10.0,0.0,NaN
2,1107,1968,SMA,Menikah,29857000.0,0.0,0.0,34.0,8092.0,22253.0,30345.0,26299.0,2.0,0.0,5.0,0.0,2013-08-06
3,4478,1971,Doktor,Menikah,117949098.0,0.0,1.0,82.0,4872.0,126061.0,0.0,9745.0,3.0,5.0,7.0,0.0,NaN
4,5080,1974,Sarjana,Rencana Menikah,164761134.0,0.0,0.0,28.0,343208.0,1416462.0,236196.0,107776.0,0.0,1.0,8.0,0.0,NaN


In [ ]:
test_feature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       3818 non-null   int64  
 1   tahun_kelahiran          3818 non-null   int64  
 2   pendidikan               3614 non-null   object 
 3   status_pernikahan        3636 non-null   object 
 4   pendapatan               3615 non-null   float64
 5   jumlah_anak_balita       3609 non-null   float64
 6   jumlah_anak_remaja       3608 non-null   float64
 7   terakhir_belanja         3617 non-null   float64
 8   belanja_buah             3632 non-null   float64
 9   belanja_daging           3623 non-null   float64
 10  belanja_ikan             3622 non-null   float64
 11  belanja_kue              3631 non-null   float64
 12  pembelian_diskon         3639 non-null   float64
 13  pembelian_web            3638 non-null   float64
 14  pembelian_toko          

In [ ]:
test_feature.drop(columns=['jumlah_anak_balita','jumlah_anak_remaja','tanggal_menjadi_anggota', 'keluhan'], inplace=True)

In [ ]:
test_feature.isnull().sum()

ID                     0
tahun_kelahiran        0
pendidikan           204
status_pernikahan    182
pendapatan           203
terakhir_belanja     201
belanja_buah         186
belanja_daging       195
belanja_ikan         196
belanja_kue          187
pembelian_diskon     179
pembelian_web        180
pembelian_toko       186
dtype: int64

In [ ]:
test_feature ['pendidikan'] = label_encoder.fit_transform(test_feature['pendidikan'])
test_feature ['status_pernikahan'] = label_encoder.fit_transform(test_feature['status_pernikahan'])

test_feature ['pendidikan'] = label_encoder.fit_transform(test_feature['pendidikan'])
test_feature ['status_pernikahan'] = label_encoder.fit_transform(test_feature['status_pernikahan'])

In [ ]:
columns_to_encode = ['pendidikan', 'status_pernikahan']

for column in columns_to_encode:
    # Menyimpan indeks di mana nilai NaN muncul
    nan_indices = test_feature[column].isnull()

    # Mengisi nilai NaN dengan string unik untuk menghindari kesalahan LabelEncoder
    test_feature[column].fillna('NaN', inplace=True)

    # Label encode kolom
    test_feature[column] = label_encoder.fit_transform(test_feature[column])

    # Kembalikan nilai NaN ke posisi semula
    test_feature[column] = np.where(nan_indices, np.nan, test_feature[column])

In [ ]:
test_feature.head()

,ID,tahun_kelahiran,pendidikan,status_pernikahan,pendapatan,terakhir_belanja,belanja_buah,belanja_daging,belanja_ikan,belanja_kue,pembelian_diskon,pembelian_web,pembelian_toko
0,2241,1957,6.0,NaN,120660151.0,63.0,122277.0,541399.0,214192.0,84305.0,4.0,6.0,10.0
1,2274,1968,1.0,3.0,163551821.0,58.0,35761.0,353335.0,63365.0,41112.0,2.0,5.0,10.0
2,1107,1968,4.0,3.0,29857000.0,34.0,8092.0,22253.0,30345.0,26299.0,2.0,0.0,5.0
3,4478,1971,1.0,3.0,117949098.0,82.0,4872.0,126061.0,0.0,9745.0,3.0,5.0,7.0
4,5080,1974,6.0,5.0,164761134.0,28.0,343208.0,1416462.0,236196.0,107776.0,0.0,1.0,8.0


In [ ]:
kolom_ID = test_feature.pop('ID')

In [ ]:
test_feature.isnull().sum()

ID                     0
tahun_kelahiran        0
pendidikan           204
status_pernikahan    182
pendapatan           203
terakhir_belanja     201
belanja_buah         186
belanja_daging       195
belanja_ikan         196
belanja_kue          187
pembelian_diskon     179
pembelian_web        180
pembelian_toko       186
dtype: int64

In [ ]:
imputer = KNNImputer(n_neighbors=5)
test_feature_imputed = imputer.fit_transform(test_feature)

In [ ]:
test_feature = pd.DataFrame(test_feature_imputed, columns = test_feature.columns)

In [ ]:
kolom_numerik_nan = test_feature.select_dtypes(exclude=['object']).columns[test_feature.select_dtypes(exclude=['object']).isnull().any()]

In [ ]:
imputer = KNNImputer(n_neighbors=5)
test_feature[kolom_numerik_nan] = imputer.fit_transform(test_feature[kolom_numerik_nan])

In [ ]:
test_feature.isnull().sum()

ID                   0
tahun_kelahiran      0
pendidikan           0
status_pernikahan    0
pendapatan           0
terakhir_belanja     0
belanja_buah         0
belanja_daging       0
belanja_ikan         0
belanja_kue          0
pembelian_diskon     0
pembelian_web        0
pembelian_toko       0
dtype: int64

In [ ]:
# 1. Identifikasi kolom-kolom kategorikal dan kolom-kolom numerik
kolom_kategorikal = test_feature.select_dtypes(include=['object']).columns
kolom_numerik = test_feature.select_dtypes(exclude=['object']).columns

In [ ]:
# 2. Pisahkan data menjadi dua bagian
data_train = test_feature.dropna(subset=kolom_kategorikal)
data_test = test_feature[test_feature[kolom_kategorikal].isnull().any(axis=1)]

In [ ]:
data_train.isnull().sum()

tahun_kelahiran      0
pendidikan           0
status_pernikahan    0
pendapatan           0
terakhir_belanja     0
belanja_buah         0
belanja_daging       0
belanja_ikan         0
belanja_kue          0
pembelian_diskon     0
pembelian_web        0
pembelian_toko       0
keluhan              0
dtype: int64

In [ ]:
# 3. Gunakan data dengan nilai yang tidak hilang untuk melatih model
X_train = data_train[kolom_numerik]
y_train = data_train[kolom_kategorikal]

In [ ]:
# 4. Gunakan model yang dilatih untuk memprediksi nilai-nilai yang hilang
model = RandomForestClassifier()  # Anda dapat menggunakan model yang lain juga
model.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
X_test = data_test[kolom_numerik]
nilai_prediksi = model.predict(X_test)

In [ ]:
# 5. Isi nilai-nilai yang hilang dengan nilai-nilai yang diprediksi
test_feature.loc[test_feature[kolom_kategorikal].isnull().any(axis=1), kolom_kategorikal] = nilai_prediksi

In [ ]:
test_feature.isnull().sum()

tahun_kelahiran      0
pendidikan           0
status_pernikahan    0
pendapatan           0
terakhir_belanja     0
belanja_buah         0
belanja_daging       0
belanja_ikan         0
belanja_kue          0
pembelian_diskon     0
pembelian_web        0
pembelian_toko       0
keluhan              0
dtype: int64

In [ ]:
test_feature = pd.concat([kolom_ID, test_feature], axis=1)

In [ ]:
test_feature.head()

,ID,tahun_kelahiran,pendidikan,status_pernikahan,pendapatan,terakhir_belanja,belanja_buah,belanja_daging,belanja_ikan,belanja_kue,pembelian_diskon,pembelian_web,pembelian_toko,keluhan
0,2241,1957,Sarjana,Menikah,120660151.0,63.0,122277.0,541399.0,214192.0,84305.0,4.0,6.0,10.0,0.0
1,2274,1968,Doktor,Menikah,163551821.0,58.0,35761.0,353335.0,63365.0,41112.0,2.0,5.0,10.0,0.0
2,1107,1968,SMA,Menikah,29857000.0,34.0,8092.0,22253.0,30345.0,26299.0,2.0,0.0,5.0,0.0
3,4478,1971,Doktor,Menikah,117949098.0,82.0,4872.0,126061.0,0.0,9745.0,3.0,5.0,7.0,0.0
4,5080,1974,Sarjana,Rencana Menikah,164761134.0,28.0,343208.0,1416462.0,236196.0,107776.0,0.0,1.0,8.0,0.0


In [ ]:
# Preprocessing
X_train = train_feature
y_train = train_labels
X_test = test_feature.drop(columns=['ID'])

In [ ]:
X_train.head()

,tahun_kelahiran,pendidikan,status_pernikahan,pendapatan,terakhir_belanja,belanja_buah,belanja_daging,belanja_ikan,belanja_kue,pembelian_diskon,pembelian_web,pembelian_toko
0,1979.0,6.0,5.0,107028212.4,27.2,50575.0,260967.0,50575.0,20230.0,2.0,2.0,5.0
1,1950.0,6.0,5.0,84063000.0,70.0,6069.0,44506.0,80920.0,20230.0,9.0,6.0,4.0
2,1966.0,6.0,3.0,127532564.0,45.0,117611.0,265460.0,96341.0,145573.0,1.0,1.0,7.0
3,1961.0,2.0,5.0,165579620.0,90.0,206346.0,1613901.0,27725.0,125868.0,0.0,7.0,8.0
4,1970.0,6.0,5.0,117703159.0,78.0,90563.0,311757.0,40358.0,33875.0,7.0,6.0,5.0


In [ ]:
X_train ['pendidikan'] = label_encoder.fit_transform(X_train['pendidikan'])
X_train ['status_pernikahan'] = label_encoder.fit_transform(X_train['status_pernikahan'])

X_test ['pendidikan'] = label_encoder.fit_transform(X_test['pendidikan'])
X_test ['status_pernikahan'] = label_encoder.fit_transform(X_test['status_pernikahan'])

In [ ]:
X_train.head()

,tahun_kelahiran,pendidikan,status_pernikahan,pendapatan,terakhir_belanja,belanja_buah,belanja_daging,belanja_ikan,belanja_kue,pembelian_diskon,pembelian_web,pembelian_toko,keluhan
0,1979,5,4,107057433.0,27.8,50575.0,260967.0,50575.0,20230.0,2.0,2.0,5.0,0.0
1,1950,5,4,84063000.0,70.0,6069.0,44506.0,80920.0,20230.0,9.0,6.0,4.0,0.0
2,1966,5,3,127532564.0,45.0,117611.0,265460.0,96341.0,145573.0,1.0,1.0,7.0,0.0
3,1961,2,4,165579620.0,90.0,206346.0,1613901.0,27725.0,125868.0,0.0,7.0,8.0,0.0
4,1970,5,4,117703159.0,78.0,90563.0,311757.0,40358.0,33875.0,7.0,6.0,5.0,0.0


In [ ]:
# Pra-pemrosesan SCALER
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#XGB
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
#DST
model = DecisionTreeClassifier()  # Inisialisasi model Decision Tree dengan parameter default
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [ ]:
# Inisialisasi dan pelatihan model AdaBoost
model = AdaBoostClassifier()  # Inisialisasi model AdaBoost dengan parameter default
model.fit(X_train, y_train)   # Pelatihan model menggunakan data pelatihan

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostClassifier()

In [ ]:
# Train model SVM
model = svm.SVC(kernel='rbf', C=1, gamma='scale')  # contoh dengan kernel RBF
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1)

In [ ]:
# Train model Random Forest
model = RandomForestClassifier()
model.fit(X_train, y_train)

<ipython-input-219-557451b6d53f>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


RandomForestClassifier()

In [ ]:
train_predictions = model.predict(X_train)

In [ ]:
f_macro = f1_score(y_train, train_predictions, average='macro')
print("Macro F-Score on validation set:", f_macro)
print("Classification Report:")
print(classification_report(y_train, train_predictions))

Macro F-Score on validation set: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       983
           1       1.00      1.00      1.00       515
           2       1.00      1.00      1.00       389
           3       1.00      1.00      1.00       472
           4       1.00      1.00      1.00       515
           5       1.00      1.00      1.00       557
           6       1.00      1.00      1.00       386

    accuracy                           1.00      3817
   macro avg       1.00      1.00      1.00      3817
weighted avg       1.00      1.00      1.00      3817



In [ ]:
test_predictions = model.predict(X_test)

In [ ]:
submission = pd.DataFrame({'ID': test_feature['ID'], 'jumlah_promosi': test_predictions})
submission['ID'] = submission['ID'].astype(int)

In [ ]:
# Save submission to CSV
submission.to_csv('submission.csv', index=False)

In [ ]:
pd.set_option('display.max_rows', None)

# Menampilkan 100 baris pertama dari DataFrame
submission.head(100)

,ID,jumlah_promosi
0,2241,0
1,2274,0
2,1107,1
3,4478,3
4,5080,4
5,5725,4
6,456,6
7,1711,0
8,2940,0
9,3031,0
